In [59]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [60]:
def corpus(data):
    data["noticia"] = data.titulo + " " + data.subTitulo + " " + data.conteudo
    docs = []

    for index, doc in data.iterrows():
        doc.noticia = doc.noticia.lower()
        docs.append(doc.noticia)

    return docs

In [61]:
data = pd.read_csv("estadao_noticias_eleicao.csv", encoding = "utf-8")[:500]
data = data.replace(np.NAN, "")

In [62]:
corpus = corpus(data)

vectorizer = TfidfVectorizer()
vector = vectorizer.fit_transform(corpus)

x = vector.toarray() #Caracteristicas

In [63]:
# K-means
kmeans = KMeans(n_clusters=2, random_state=0)
kmeans.fit(vector.toarray()) # passando as caracteristicas para o treinamento

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=2, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=0, tol=0.0001, verbose=0)

In [64]:
len(kmeans.labels_) # grupos criados

500

In [56]:
dt = {}
i = 0
for v in x:
    dt[i] = np.array(v)
    i += 1

In [76]:
dt

{0: array([0., 0., 0., ..., 0., 0., 0.]),
 1: array([0., 0., 0., ..., 0., 0., 0.]),
 2: array([0., 0., 0., ..., 0., 0., 0.]),
 3: array([0., 0., 0., ..., 0., 0., 0.]),
 4: array([0., 0., 0., ..., 0., 0., 0.]),
 5: array([0., 0., 0., ..., 0., 0., 0.]),
 6: array([0.        , 0.00749098, 0.        , ..., 0.        , 0.        ,
        0.        ]),
 7: array([0., 0., 0., ..., 0., 0., 0.]),
 8: array([0., 0., 0., ..., 0., 0., 0.]),
 9: array([0.        , 0.05588102, 0.        , ..., 0.        , 0.        ,
        0.        ]),
 10: array([0., 0., 0., ..., 0., 0., 0.]),
 11: array([0., 0., 0., ..., 0., 0., 0.]),
 12: array([0., 0., 0., ..., 0., 0., 0.]),
 13: array([0., 0., 0., ..., 0., 0., 0.]),
 14: array([0., 0., 0., ..., 0., 0., 0.]),
 15: array([0., 0., 0., ..., 0., 0., 0.]),
 16: array([0., 0., 0., ..., 0., 0., 0.]),
 17: array([0., 0., 0., ..., 0., 0., 0.]),
 18: array([0., 0., 0., ..., 0., 0., 0.]),
 19: array([0., 0., 0., ..., 0., 0., 0.]),
 20: array([0., 0., 0., ..., 0., 0., 0

In [81]:
#dframe = pd.DataFrame(dt, [i for i in range(17214)])
for i in range(dt.keys()):
    t = pd.Series([j for j in dt[i]], index = [i])

ValueError: Wrong number of items passed 17214, placement implies 1

In [79]:
#dframe['k-labels'] = kmeans.labels_
dframe

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.007491,0.000000,0.0,0.055881,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.006449,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
9,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000
